In [19]:
import itertools
from operator import itemgetter

import pandas as pd
import geopandas as gpd
import numpy as np
import pygeos

# from scipy.spatial import cKDTree

In [2]:
HISTORICAL_FIRES_RAW_SOURCE = '/Users/awilliam/Desktop/wps/api/PROT_HISTORICAL_INCIDENTS_SP.geojson'
RAILWAYS_RAW_SOURCE = '/Users/awilliam/Desktop/wps/api/GBA_RAILWAY_TRACKS_SP.geojson'

In [3]:
with open(HISTORICAL_FIRES_RAW_SOURCE) as file:
    df = gpd.read_file(file)

In [4]:
display(df)

,FIRE_NUMBER,FIRE_YEAR,IGNITION_DATE,FIRE_CAUSE,FIRE_LABEL,FIRE_CENTRE,ZONE,FIRE_ID,FIRE_TYPE,GEOGRAPHIC_DESCRIPTION,LATITUDE,LONGITUDE,CURRENT_SIZE,FEATURE_CODE,OBJECTID,SE_ANNO_CAD_DATA,fme_feature_type,geometry
0,K50756,2013,20130818091300,Person,2013-K50756,5.0,5.0,500756,Nuisance Fire,Darke Lake,49.7118,-119.8661,0.009,JA70003000,14704564,None,WHSE_LAND_AND_NATURAL_RESOURCE.PROT_HISTORICAL...,POINT (-119.86608 49.71180)
1,G90349,2004,20040717150300,Lightning,2004-G90349,4.0,9.0,400349,Fire,Thetlaandoa Creek,59.5515,-121.4590,1.000,JA70003000,14704565,None,WHSE_LAND_AND_NATURAL_RESOURCE.PROT_HISTORICAL...,POINT (-121.45900 59.55150)
2,G13106,2018,20181010155200,Unknown,2018-G13106,4.0,1.0,403106,Smoke Chase,salmon valley,54.2382,-122.6071,NaN,JA70003000,14704566,None,WHSE_LAND_AND_NATURAL_RESOURCE.PROT_HISTORICAL...,POINT (-122.60712 54.23817)
3,K61162,2002,20020923124900,Unknown,2002-K61162,5.0,6.0,501162,Smoke Chase,S End of Peter Hope Lk.,50.2859,-120.3123,NaN,JA70003000,14704567,None,WHSE_LAND_AND_NATURAL_RESOURCE.PROT_HISTORICAL...,POINT (-120.31232 50.28593)
4,N70289,2002,20020723154000,Lightning,2002-N70289,6.0,7.0,600289,Fire,Above Ainsworth,49.7512,-116.9222,0.009,JA70003000,14704568,None,WHSE_LAND_AND_NATURAL_RESOURCE.PROT_HISTORICAL...,POINT (-116.92217 49.75123)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187142,K60287,1998,19980729,Lightning,1998-K60287,5.0,6.0,287,Fire,Dry Lake #3,49.6420,-120.6655,0.100,JA70003000,14888926,None,WHSE_LAND_AND_NATURAL_RESOURCE.PROT_HISTORICAL...,POINT (-120.66550 49.64200)
187143,K60288,1998,19980728,Lightning,1998-K60288,5.0,6.0,288,Fire,AP Ranch,49.7584,-120.6811,0.100,JA70003000,14888927,None,WHSE_LAND_AND_NATURAL_RESOURCE.PROT_HISTORICAL...,POINT (-120.68108 49.75837)
187144,K60302,1998,19980728,Lightning,1998-K60302,5.0,6.0,302,Fire,S of Airport Ranch,49.7384,-120.6289,0.100,JA70003000,14888928,None,WHSE_LAND_AND_NATURAL_RESOURCE.PROT_HISTORICAL...,POINT (-120.62892 49.73837)
187145,K60341,1998,19980731,Lightning,1998-K60341,5.0,6.0,341,Fire,Pimanus Hill,50.3798,-121.1803,0.100,JA70003000,14888929,None,WHSE_LAND_AND_NATURAL_RESOURCE.PROT_HISTORICAL...,POINT (-121.18033 50.37983)


In [5]:
df = df[df.FIRE_CAUSE == 'Person']
display(df)

,FIRE_NUMBER,FIRE_YEAR,IGNITION_DATE,FIRE_CAUSE,FIRE_LABEL,FIRE_CENTRE,ZONE,FIRE_ID,FIRE_TYPE,GEOGRAPHIC_DESCRIPTION,LATITUDE,LONGITUDE,CURRENT_SIZE,FEATURE_CODE,OBJECTID,SE_ANNO_CAD_DATA,fme_feature_type,geometry
0,K50756,2013,20130818091300,Person,2013-K50756,5.0,5.0,500756,Nuisance Fire,Darke Lake,49.7118,-119.8661,0.009,JA70003000,14704564,None,WHSE_LAND_AND_NATURAL_RESOURCE.PROT_HISTORICAL...,POINT (-119.86608 49.71180)
7,K40034,2000,20000430150000,Person,2000-K40034,5.0,4.0,500034,Fire,1 Km South of Vernon,50.2110,-119.3833,0.010,JA70003000,14704619,None,WHSE_LAND_AND_NATURAL_RESOURCE.PROT_HISTORICAL...,POINT (-119.38333 50.21103)
8,V10252,2013,20130721144900,Person,2013-V10252,2.0,1.0,200252,Nuisance Fire,Chilliwack River 1 km Mark,49.1041,-121.6512,0.000,JA70003000,14704620,None,WHSE_LAND_AND_NATURAL_RESOURCE.PROT_HISTORICAL...,POINT (-121.65117 49.10413)
9,K20135,2003,20030705120000,Person,2003-K20135,5.0,2.0,500135,Fire,Isobel Lake,50.8369,-120.4142,0.100,JA70003000,14704621,None,WHSE_LAND_AND_NATURAL_RESOURCE.PROT_HISTORICAL...,POINT (-120.41417 50.83693)
10,K51159,2004,20040626102000,Person,2004-K51159,5.0,5.0,501159,Nuisance Fire,James Lake,49.9535,-119.2491,NaN,JA70003000,14704622,None,WHSE_LAND_AND_NATURAL_RESOURCE.PROT_HISTORICAL...,POINT (-119.24910 49.95350)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187114,K60162,1998,19980714,Person,1998-K60162,5.0,6.0,162,Fire,8 Km S of Princeton,49.4238,-120.5464,0.100,JA70003000,14888898,None,WHSE_LAND_AND_NATURAL_RESOURCE.PROT_HISTORICAL...,POINT (-120.54638 49.42380)
187115,K60164,1998,19980714,Person,1998-K60164,5.0,6.0,164,Fire,Lilly Lake Rd,50.0746,-120.8138,0.100,JA70003000,14888899,None,WHSE_LAND_AND_NATURAL_RESOURCE.PROT_HISTORICAL...,POINT (-120.81380 50.07458)
187116,K60172,1998,19980719,Person,1998-K60172,5.0,6.0,172,Fire,W. of Hedley,49.3728,-120.1402,0.200,JA70003000,14888900,None,WHSE_LAND_AND_NATURAL_RESOURCE.PROT_HISTORICAL...,POINT (-120.14015 49.37278)
187117,K60206,1998,19980727,Person,1998-K60206,5.0,6.0,206,Fire,Peterson Road,49.9568,-120.9268,0.100,JA70003000,14888901,None,WHSE_LAND_AND_NATURAL_RESOURCE.PROT_HISTORICAL...,POINT (-120.92678 49.95677)


In [6]:
with open(RAILWAYS_RAW_SOURCE) as file:
    railways_df = gpd.read_file(file)
display(railways_df)

,RAILWAY_TRACK_ID,NID,TRACK_SEGMENT_ID,TRACK_NAME,TRACK_CLASSIFICATION,REGULATOR,TRANSPORT_TYPE,USE_TYPE,GAUGE,NUMBER_OF_TRACKS,...,ATTRIBUTE_CREATION_DATE,ATTRIBUTE_REVISION_DATE,ATTRIBUTE_ACQUISITION_TECHNQUE,ATTRIBUTE_PROVIDER,OBJECTID,SE_ANNO_CAD_DATA,FEATURE_LENGTH_M,SHAPE.LEN,fme_feature_type,geometry
0,32,2d3aad84df1d4dc099bc5b4eb215980b,529c791850a143b0963735ba24ff0fe0,None,Yard,Unknown,Train,Freight,Standard,1,...,20131201,20170101,Vector Data,Federal,2200530,None,599.7485,0,WHSE_BASEMAPPING.GBA_RAILWAY_TRACKS_SP,"LINESTRING (-122.88502 49.23680, -122.88533 49..."
1,33,46a7b6be1d0c4b1090450f2b221844bd,0da02473327c4594b9eed7c3dec5e21e,Cranbrook,Yard,Unknown,Train,Freight,Standard,1,...,20131201,20170101,Vector Data,Federal,2200531,None,164.8002,0,WHSE_BASEMAPPING.GBA_RAILWAY_TRACKS_SP,"LINESTRING (-115.79600 49.49847, -115.79559 49..."
2,34,33a1254f55af46308eb3e568b25d27b8,84264d99fbf24c339e48c9821c391f76,None,Spur,Unknown,Train,Freight,Standard,1,...,20170101,20170101,Vector Data,Federal,2200532,None,552.6092,0,WHSE_BASEMAPPING.GBA_RAILWAY_TRACKS_SP,"LINESTRING (-122.26817 49.01137, -122.26820 49..."
3,35,d09b96659c43418fb7010f70cb3319e3,ddc213aabbc542a6a5cc4e8f748f6944,None,Yard,Unknown,Train,Freight,Standard,1,...,20131201,20170101,Vector Data,Federal,2200533,None,28.0812,0,WHSE_BASEMAPPING.GBA_RAILWAY_TRACKS_SP,"LINESTRING (-122.83434 49.21688, -122.83472 49..."
4,36,28e5539e1b1f416bbb1b3826bf2a5264,b48912aadaa54b69b7605dd8d29cbafe,Iris Spur No. 6007,Spur,Unknown,Train,Freight,Standard,1,...,20131201,20170101,Vector Data,Federal,2200534,None,491.9740,0,WHSE_BASEMAPPING.GBA_RAILWAY_TRACKS_SP,"LINESTRING (-115.80699 50.15382, -115.80806 50..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10039,9391,a6623c015bb64bcabb46f61564008c59,c046f29fea4345cb8ea0dc151b9e62e4,Yard Lead,Yard,Unknown,Train,Freight,Standard,1,...,20170101,20170101,Vector Data,Federal,2209881,None,421.2427,0,WHSE_BASEMAPPING.GBA_RAILWAY_TRACKS_SP,"LINESTRING (-116.95783 51.28794, -116.95678 51..."
10040,9392,53f0a3076a424a3c932c6f457c501225,a104164a4ddd4dc7b9859ae3cd4c5b0e,None,Main,Unknown,Train,Freight and Tourist,Standard,1,...,20131201,20170101,Vector Data,Federal,2209882,None,110.0708,0,WHSE_BASEMAPPING.GBA_RAILWAY_TRACKS_SP,"LINESTRING (-122.74817 53.88362, -122.74818 53..."
10041,9393,feb8fc60c2f44c82aad7d84cdf8411a1,c0b41508cdad45c8a1d3b98c04ecb15f,Morice Town,Siding,Unknown,Train,Freight and Passenger,Standard,1,...,20170101,20170101,Vector Data,Federal,2209883,None,2349.0872,0,WHSE_BASEMAPPING.GBA_RAILWAY_TRACKS_SP,"LINESTRING (-127.34197 55.03751, -127.34112 55..."
10042,9394,ea5734d66dd34e849f5635f3fc458c9d,0c0c61b58c9f4112b18a36c63217ba93,LE40 Runaround,Siding,Unknown,Train,Freight,Standard,1,...,20170101,20170101,Vector Data,Federal,2209884,None,402.4594,0,WHSE_BASEMAPPING.GBA_RAILWAY_TRACKS_SP,"LINESTRING (-119.03470 50.23469, -119.03396 50..."


In [16]:
railways_df['geometry'] = railways_df['geometry'].to_crs(epsg=3035)
df['geometry'] = df['geometry'].to_crs(epsg=3035)
fires_and_railways = df.sjoin_nearest(railways_df, distance_col="distance_to_railway")
# distance is in metres
display(fires_and_railways)

,FIRE_NUMBER,FIRE_YEAR,IGNITION_DATE,FIRE_CAUSE,FIRE_LABEL,FIRE_CENTRE,ZONE,FIRE_ID,FIRE_TYPE,GEOGRAPHIC_DESCRIPTION,...,ATTRIBUTE_CREATION_DATE,ATTRIBUTE_REVISION_DATE,ATTRIBUTE_ACQUISITION_TECHNQUE,ATTRIBUTE_PROVIDER,OBJECTID_right,SE_ANNO_CAD_DATA_right,FEATURE_LENGTH_M,SHAPE.LEN,fme_feature_type_right,distance_to_railway
0,K50756,2013,20130818091300,Person,2013-K50756,5.0,5.0,500756,Nuisance Fire,Darke Lake,...,20170101,20170101,Vector Data,Federal,2201495,None,6229.0755,0,WHSE_BASEMAPPING.GBA_RAILWAY_TRACKS_SP,59542.323411
88,K51091,2007,20070807110300,Person,2007-K51091,5.0,5.0,501091,Nuisance Fire,Jack Pine Lake,...,20170101,20170101,Vector Data,Federal,2201495,None,6229.0755,0,WHSE_BASEMAPPING.GBA_RAILWAY_TRACKS_SP,44744.070156
112,K50510,2000,20000806134000,Person,2000-K50510,5.0,5.0,500510,Nuisance Fire,Red Bridge,...,20170101,20170101,Vector Data,Federal,2201495,None,6229.0755,0,WHSE_BASEMAPPING.GBA_RAILWAY_TRACKS_SP,106504.162777
127,K40748,2005,20050814080000,Person,2005-K40748,5.0,4.0,500748,Nuisance Fire,Coldstream Creek,...,20170101,20170101,Vector Data,Federal,2201495,None,6229.0755,0,WHSE_BASEMAPPING.GBA_RAILWAY_TRACKS_SP,280.463163
142,K50826,2006,20060805102000,Person,2006-K50826,5.0,5.0,500826,Nuisance Fire,11km Bear Creek,...,20170101,20170101,Vector Data,Federal,2201495,None,6229.0755,0,WHSE_BASEMAPPING.GBA_RAILWAY_TRACKS_SP,29241.717673
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186735,V30013,1997,19970519,Person,1997-V30013,2.0,3.0,13,Fire,None,...,20131201,20170101,Vector Data,Federal,2200920,None,388.6699,0,WHSE_BASEMAPPING.GBA_RAILWAY_TRACKS_SP,629.167140
185467,G10164,1998,19980626,Person,1998-G10164,4.0,1.0,164,Fire,Chubb Lake,...,20170101,20170101,Vector Data,Federal,2202979,None,753.1488,0,WHSE_BASEMAPPING.GBA_RAILWAY_TRACKS_SP,3536.813558
185946,G10531,1998,19980916,Person,1998-G10531,4.0,1.0,531,Fire,Chubb Lake,...,20170101,20170101,Vector Data,Federal,2202979,None,753.1488,0,WHSE_BASEMAPPING.GBA_RAILWAY_TRACKS_SP,3536.813558
186088,N40170,1997,19970813,Person,1997-N40170,6.0,4.0,170,Fire,None,...,20170101,20170101,Vector Data,Federal,2201992,None,340.1990,0,WHSE_BASEMAPPING.GBA_RAILWAY_TRACKS_SP,482.679212


In [18]:
print(fires_and_railways['distance_to_railway'].max())
print(fires_and_railways['distance_to_railway'].min())

686888.3279566616
0.035714690821438806
